# 🧠 AIGIS - RAG Pipeline (Free Colab Version)
Retrieval-Augmented Generation using FAISS + MiniLM + Mistral 7B

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

folder_path = "knowledgebase"
all_documents = []

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        try:
            print(f"📄 Loading: {filename}")
            pdf_path = os.path.join(folder_path, filename)
            loader = PyPDFLoader(pdf_path)
            pages = loader.load()
            chunks = splitter.split_documents(pages)
            all_documents.extend(chunks)
        except Exception as e:
            print(f"❌ Failed to load {filename}: {e}")


print(f"✅ Loaded and chunked {len(all_documents)} document chunks.")


📄 Loading: A Quick Parents' guide.pdf
📄 Loading: ai-and-child-safety-online-guide-parents_caregiver.pdf
📄 Loading: Child Online Protection in and through Digital Learning.pdf
📄 Loading: CSO Checklist for online safety_3.pdf
📄 Loading: ILF-general-safety-guide.pdf


incorrect startxref pointer(4)
parsing for Object Streams
Object 40 0 not defined.


❌ Failed to load ILF-general-safety-guide.pdf: Invalid object in /Pages
📄 Loading: Industry-Guidelines-for-Online-ChildProtection.pdf
📄 Loading: Online Safety Resources.pdf
📄 Loading: Safeguarding Your Child In The Digital Age.pdf
📄 Loading: Safeguarding_Guidance to Keep Children Safe Online Eng (002).pdf
📄 Loading: Sub-report-Digital-2024.pdf
✅ Loaded and chunked 628 document chunks.


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain_core.documents import Document  # Just for test
documents = [Document(page_content="Internet safety for children is important.")]

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)


RuntimeError: Failed to import transformers.trainer_callback because of the following error (look up to see its traceback):
cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (d:\OneDrive - ESPRIT\Desktop\AIGIS\extension\aigisvenv\Lib\site-packages\huggingface_hub\__init__.py)

In [ ]:
query = "What should a parent do if hate speech and deepfake audio are detected online?"
relevant_docs = vectorstore.similarity_search(query, k=3)
context = "\n\n".join([doc.page_content for doc in relevant_docs])

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

In [ ]:
prompt = f"""Context:\n{context}\n\nQuestion: {query}\nAnswer in a professional tone:\n"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=300)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n=== Generated Report ===\n")
print(result)